In [1]:
#Import Basic Libraries
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

# Data Processing

In [2]:
#Read CSV
data = pd.read_csv("/kaggle/input/digit-recognizer/train.csv")


In [3]:
#Pandas to Numpy Array
data = np.array(data)
m,n = data.shape
np.random.shuffle(data)

In [4]:
#Validation Set
data_val = data[0:1000].T
x_val = data_val[1:n]
y_val = data_val[0]

x_val = x_val / 255

In [5]:
#Training Set
data_train = data[1000:m].T
x_train = data_train[1:n]
y_train = data_train[0]

x_train = x_train / 255

# Creating Neural Net

**Initializing Neural Net**

In [6]:
#Weights and Biases
def init_params():    
    w1 = np.random.rand(10, 784) - 0.5
    b1 = np.random.rand(10, 1) - 0.5
    w2 = np.random.rand(10, 10) - 0.5
    b2 = np.random.rand(10, 1) - 0.5
    
    return w1, b1, w2, b2


**Prediction**

In [7]:
#ReLU
def relu(z):
    return np.maximum(0, z)

In [8]:
#Softmax
def softmax(z):
    a = np.exp(z) / sum(np.exp(z))
    return a

In [9]:
#Forward Propogation
def frwd_prop(w1, b1, w2, b2, x):
    z1 = w1.dot(x) + b1
    a1 = relu(z1)
    
    z2 = w2.dot(a1) + b2
    a2 = softmax(z2)
    
    return z1, a1, z2, a2

**Loss**

In [10]:
#1-Hot Encoder
def one_hot(y):
    one_hot_y = np.zeros((y.size, y.max() + 1))
    one_hot_y[np.arange(y.size), y] = 1
    return one_hot_y.T


In [11]:
#Derivative of ReLU
def d_relu(z):
    return z > 0

In [12]:
#Back Propogation
def back_prop(z1, a1, z2, a2, w1, w2, x, y):
    m = y.size
    one_hot_y = one_hot(y)
    
    dz2 = a2 - one_hot_y
    dw2 = (1/m) * dz2.dot(a1.T)
    db2 = (1/m) * np.sum(dz2)
    
    dz1 = w2.T.dot(dz2) * d_relu(z1)
    dw1 = (1/m) * dz1.dot(x.T)
    db1 = (1/m) * np.sum(dz1)
    
    return dw1, db1, dw2, db2

**Updating Neural Net**

In [13]:
#Update Parameters
def update_params(alpha, w1, w2, b1, b2, dw1, dw2, db1, db2):
    w1 = w1 - alpha*dw1
    b1 = b1 - alpha*db1
    w2 = w2 - alpha*dw2
    b2 = b2 - alpha*db2
    
    return w1, b1, w2, b2

# Gradient Descent Algorithm

In [14]:
#Prediction
def get_pred(a2):
    return np.argmax(a2, 0)

In [15]:
#Accuracy
def get_accu(pred, y):
    return np.sum(pred == y) / y.size

In [16]:
#Gradient Descent
def grad_desc(x, y, iterations, alpha):
    w1, b1, w2, b2 = init_params()
    for i in range(iterations):
        z1, a1, z2, a2 = frwd_prop(w1, b1, w2, b2, x)
        dw1, db1, dw2, db2 = back_prop(z1, a1, z2, a2, w1, w2, x, y)
        w1, b1, w2, b2 = update_params(alpha, w1, w2, b1, b2, dw1, dw2, db1, db2)
        
        if i % 50 == 0:
            print("Iterations: ", i)
            print("Accuracy: ", get_accu(get_pred(a2), y))
            
    return w1, b1, w2, b2
        

# Training & Validation

In [17]:
#Training
w1, b1, w2, b2 = grad_desc(x_train, y_train, 500, 0.1)

Iterations:  0
Accuracy:  0.09580487804878049
Iterations:  50
Accuracy:  0.4091951219512195
Iterations:  100
Accuracy:  0.624
Iterations:  150
Accuracy:  0.7126585365853658
Iterations:  200
Accuracy:  0.7585365853658537
Iterations:  250
Accuracy:  0.7853658536585366
Iterations:  300
Accuracy:  0.8033902439024391
Iterations:  350
Accuracy:  0.8169512195121951
Iterations:  400
Accuracy:  0.827780487804878
Iterations:  450
Accuracy:  0.8361463414634146


*~ 85% Accuracy*

In [18]:
#Testing Function
def make_pred(x, w1, b1, w2, b2):
    _, _, _, a2 = frwd_prop(w1, b1, w2, b2, x)
    pred = get_pred(a2)
    return pred

In [19]:
#Validation
pred = make_pred(x_val, w1, b1, w2, b2)
accu = get_accu(pred, y_val)

accu

0.846

*~ 85% Accuracy, Model is Generalised*

# Testing

In [20]:
test_set = pd.read_csv("/kaggle/input/digit-recognizer/test.csv")
pred = make_pred(x_val, w1, b1, w2, b2)
accu = get_accu(pred, y_val)

accu

0.846

*~ 85% Accuracy*